In [78]:
import os
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import rasterio
import hydromt

from pathlib import Path
from osgeo import gdal
from hydromt_sfincs import SfincsModel
from rasterio.mask import mask
from rasterio.features import rasterize
from rasterio.transform import from_origin

### Models
Load all models and results and add them to a dictionary:

In [44]:
model_names = os.listdir(Path("model"))
models = {}
results = {}
for model_name in model_names:
    model = SfincsModel(fr"model/{model_name}/", mode="r")
    model.read_results()
    
    models[model_name] = model
    results[model_name] = model.results

In [67]:
# We append 1 model as the base model, with a representative grid.
mod0 = models["garissa_depth3m_fabdem_grid100"]
print(mod0.grid)

<xarray.Dataset> Size: 465kB
Dimensions:      (y: 243, x: 211)
Coordinates:
  * y            (y) float64 2kB -6.356e+04 -6.346e+04 ... -3.946e+04 -3.936e+04
  * x            (x) float64 2kB 4.402e+06 4.403e+06 ... 4.423e+06 4.423e+06
    spatial_ref  int32 4B 0
Data variables:
    msk          (y, x) uint8 51kB 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    scs          (y, x) float32 205kB -9.999e+03 -9.999e+03 ... -9.999e+03
    dep          (y, x) float32 205kB nan nan nan nan nan ... nan nan nan nan
Attributes:
    _FillValue:  0


### Flood maps
Load and plot the available flood maps:

In [23]:
# Available floodmaps
floodmap_names = {
"opera":[
    "2023-11-11",
    "2023-11-21",
    "2023-11-29",
],
"unosat":[
    "2023-11-26",
    "2023-12-01",
]}
print(floodmap_names)

{'opera': ['2023-11-11', '2023-11-21', '2023-11-29'], 'unosat': ['2023-11-26', '2023-12-01']}


In [25]:
analysis_extent_path = Path(r"data/2_floodmaps/analysis_extent_garissa.shp")
analysis_extent = gpd.read_file(analysis_extent_path)

### Raster floodmap
OPERA has a raster format.

In [68]:
floodmaps_opera = {}
for date in floodmap_names["opera"]:
    path = Path(f"data/2_floodmaps/opera/{date}/floodmap.tif")

    floodmap = hydromt.open_raster(path).load().astype(np.int8)
    floodmap.raster.set_nodata(-1)
    floodmap.raster.set_spatial_dims(x_dim="x", y_dim="y")
    floodmap = floodmap.raster.reproject_like(mod0.grid, method='nearest')
    floodmap.raster.to_raster(Path(f"data/2_floodmaps/reprojected_maps/opera_{date}.tif"))

    floodmaps_opera[date] = floodmap    

<xarray.Dataset> Size: 4B
Dimensions:      ()
Coordinates:
    spatial_ref  int32 4B 0
Data variables:
    *empty*
<xarray.Dataset> Size: 4B
Dimensions:      ()
Coordinates:
    spatial_ref  int32 4B 0
Data variables:
    *empty*
<xarray.Dataset> Size: 4B
Dimensions:      ()
Coordinates:
    spatial_ref  int32 4B 0
Data variables:
    *empty*


### Vector floodmap
UNOSAT has a vector format. Sentinel-2 image was acquired on 26 November 2023 at 10:50 local time. Observation time on 1 December 2023 is unknown.

In [89]:
floodmaps_unosat = {}

for date in floodmap_names["unosat"]:
    path = Path(f"data/2_floodmaps/unosat/{date}/floodmap_raster_qgis.tif")
    
    floodmap = hydromt.open_raster(path).load().astype(np.int8)
    floodmap.raster.set_nodata(-1)
    floodmap.raster.set_spatial_dims(x_dim="x", y_dim="y")
    floodmap = floodmap.raster.reproject_like(mod0.grid, method='nearest')
    floodmap.raster.to_raster(Path(f"data/2_floodmaps/reprojected_maps/unosat_{date}.tif"))

    floodmaps_unosat[date] = floodmap

{'y': <xarray.DataArray 'y' (y: 10000)> Size: 80kB
array([-0.398445, -0.398458, -0.398471, ..., -0.526447, -0.52646 , -0.526472])
Coordinates:
  * y            (y) float64 80kB -0.3984 -0.3985 -0.3985 ... -0.5265 -0.5265
    spatial_ref  int32 4B 0, 'x': <xarray.DataArray 'x' (x: 10000)> Size: 80kB
array([39.592442, 39.592452, 39.592462, ..., 39.691702, 39.691712, 39.691722])
Coordinates:
  * x            (x) float64 80kB 39.59 39.59 39.59 39.59 ... 39.69 39.69 39.69
    spatial_ref  int32 4B 0}
{'y': <xarray.DataArray 'y' (y: 10000)> Size: 80kB
array([-0.398304, -0.398317, -0.39833 , ..., -0.530168, -0.530181, -0.530194])
Coordinates:
  * y            (y) float64 80kB -0.3983 -0.3983 -0.3983 ... -0.5302 -0.5302
    spatial_ref  int32 4B 0, 'x': <xarray.DataArray 'x' (x: 10000)> Size: 80kB
array([39.593347, 39.593355, 39.593364, ..., 39.682432, 39.682441, 39.68245 ])
Coordinates:
  * x            (x) float64 80kB 39.59 39.59 39.59 39.59 ... 39.68 39.68 39.68
    spatial_ref  int32 4B 0

In [77]:
# xmin = mod0.grid.x.min()
# ymax = mod0.grid.y.max()
# nx_mod0 = int(mod0.grid.x.shape[0])*10
# ny_mod0 = int(mod0.grid.y.shape[0])*10

# transform = from_origin(xmin, ymax, nx_mod0, ny_mod0)

# # Firstly, we rasterize the vector to a grid 100 times finer than our model grid.
# floodmap_vector = gpd.read_file(Path(f"data/2_floodmaps/unosat/{date}/floodmap.shp"))

# floodmap_raster = rasterize(
#     [(geom, 1) for geom in floodmap_vector.geometry], 
#     out_shape=(ny_mod0, nx_mod0),
#     transform=transform,    # Transform to define raster georeferencing
#     fill=0,                 # Fill value for pixels outside polygons
#     all_touched=True,       # Use all pixels touched by geometries
#     dtype='uint8'
# )
# # Save this raster to a .tif file, so we can open it again with hydromt.
# with rasterio.open(Path(f"data/2_floodmaps/unosat/{date}/floodmap_raster.tif"), 'w', driver='GTiff',
#                height=floodmap_raster.shape[0], width=floodmap_raster.shape[1],
#                count=1, dtype=floodmap_raster.dtype,
#                crs=floodmap_vector.crs, transform=transform) as dst:
#     dst.write(floodmap_raster, 1)


24300